In [1]:
from access import Access, weights, Datasets
from shapely.geometry import Point
import logging
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

## Input Datasets
Load the population data and facility data from csv files

In [2]:
pop = pd.read_csv('质心.csv')
des = pd.read_csv('destination_4types.csv')
merg = pd.read_csv('merged_cleaned.csv')

Overview of the data

In [3]:
pop.head()

,objectid,lad19cd,lad19nm,lad19nmw,bng_e,bng_n,long,lat,st_areasha,st_lengths
0,1,E06000001,Hartlepool,NaN,447160,531474,-1.259401,54.669450,9.371262e+07,71011.933949
1,2,E06000002,Middlesbrough,NaN,451141,516887,-1.222262,54.542004,5.388156e+07,44481.691242
2,3,E06000003,Redcar and Cleveland,NaN,464361,519597,-1.020684,54.551652,2.450695e+08,96703.989701
3,4,E06000004,Stockton-on-Tees,NaN,444940,518183,-1.332280,54.561613,2.049330e+08,123408.985928
4,5,E06000005,Darlington,NaN,428029,515648,-1.552628,54.548705,1.974757e+08,107206.401677


In [4]:
des.head()

,objectid,Region,STP,Trust or Site Name,Address,Postcode,longitude,latitude,category
0,1,South West,NHS Kernow CCG,Helston Medical Centre,"Trelawny Road, Helston, Cornwall",TR13 8AU,-5.269119,50.099159,GP_led vaccination services
1,2,South West,NHS Kernow CCG,Isles of Scilly Health Centre (deliver to main...,"The Health Centre, St Mary's, via Helston Medi...",TR21 0HE,-5.269998,50.099710,GP_led vaccination services
2,3,South West,NHS Kernow CCG,Falmouth Health Centre,"Falmouth Health Centre, Trescobeas Rd, Falmouth",TR11 2LH,-5.086106,50.155956,GP_led vaccination services
3,4,South West,NHS Kernow CCG,Bodriggy Surgery,Bodriggy Health Centre,TR27 4PB,-5.412844,50.187164,GP_led vaccination services
4,5,South West,Cornwall and the Isles of Scilly,Truro Showground,"The Showground, Tubbin Hill, Stithians, Truro",TR3 7DP,-5.164745,50.194484,vaccination centres


In [5]:
merg.head()

,object_id,Region,CCG,Site_name,Address,Postcode,longitude,latitude,certain_or,coordinate,error,STP,Trust_or_S,Field9,Field10,Field11,STP_ICS,Centre,layer,path
0,1.0,East of England,NHS Basildon and Brentwood CCG,Brentwood Centre,"Doddinghurst Road, Pilgrims Hatch, Brentwood",CM15 9NN,0.304934,51.642242,GEOMETRIC_CENTER,wgs84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GP_led_vaccination_services,D:/access/shp_vac_sites/GP_led_vaccination_ser...
1,2.0,East of England,NHS Basildon and Brentwood CCG,Emmanuel Church Hall,"Laindon Road, Billericay",CM12 9LD,0.416827,51.617519,GEOMETRIC_CENTER,wgs84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GP_led_vaccination_services,D:/access/shp_vac_sites/GP_led_vaccination_ser...
2,3.0,East of England,NHS Basildon and Brentwood CCG,George Hurd Centre,"Audley Way, Basildon",SS14 2FL,0.460307,51.572952,GEOMETRIC_CENTER,wgs84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GP_led_vaccination_services,D:/access/shp_vac_sites/GP_led_vaccination_ser...
3,4.0,East of England,NHS Basildon and Brentwood CCG,The Nevendon Centre,"Nevendon Road, Wickford",SS12 0QG,0.515221,51.601814,GEOMETRIC_CENTER,wgs84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GP_led_vaccination_services,D:/access/shp_vac_sites/GP_led_vaccination_ser...
4,5.0,East of England,NHS Bedfordshire CCG,Guru Nanak Gurdwara,"84 Ford End Road, Bedford",MK40 4JX,-0.485502,52.131068,ROOFTOP,wgs84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GP_led_vaccination_services,D:/access/shp_vac_sites/GP_led_vaccination_ser...


Merge two facility data and give every row an ID 

In [6]:
des = pd.concat([des,merg], ignore_index=True, sort=False)
des['new_id'] = des.index
des

,objectid,Region,STP,Trust or Site Name,Address,Postcode,longitude,latitude,category,object_id,...,error,Trust_or_S,Field9,Field10,Field11,STP_ICS,Centre,layer,path,new_id
0,1.0,South West,NHS Kernow CCG,Helston Medical Centre,"Trelawny Road, Helston, Cornwall",TR13 8AU,-5.269119,50.099159,GP_led vaccination services,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2.0,South West,NHS Kernow CCG,Isles of Scilly Health Centre (deliver to main...,"The Health Centre, St Mary's, via Helston Medi...",TR21 0HE,-5.269998,50.099710,GP_led vaccination services,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,3.0,South West,NHS Kernow CCG,Falmouth Health Centre,"Falmouth Health Centre, Trescobeas Rd, Falmouth",TR11 2LH,-5.086106,50.155956,GP_led vaccination services,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,4.0,South West,NHS Kernow CCG,Bodriggy Surgery,Bodriggy Health Centre,TR27 4PB,-5.412844,50.187164,GP_led vaccination services,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,5.0,South West,Cornwall and the Isles of Scilly,Truro Showground,"The Showground, Tubbin Hill, Stithians, Truro",TR3 7DP,-5.164745,50.194484,vaccination centres,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3479,NaN,South West,NaN,NaN,NaN,PL2 3DQ,-4.151038,50.388252,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Devon,Home Park,vaccination_center,D:/access/shp_vac_sites/vaccination_center.shp,3479
3480,NaN,South West,NaN,NaN,NaN,EX5 1DJ,-3.440544,50.713022,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Devon,WestPoint Centre Ltd,vaccination_center,D:/access/shp_vac_sites/vaccination_center.shp,3480
3481,NaN,South West,NaN,NaN,NaN,BH2 5BH,-1.879189,50.717658,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Dorset,Bournemouth International Centre,vaccination_center,D:/access/shp_vac_sites/vaccination_center.shp,3481
3482,NaN,South West,NaN,NaN,NaN,TA3 7BL,-3.079995,50.987767,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Somerset,Taunton Racecourse,vaccination_center,D:/access/shp_vac_sites/vaccination_center.shp,3482


## Transform the dataframe data geodataframe
get the coordinates of the point to generate shp

In [7]:
geometry = [Point(xy) for xy in zip(pop.long, pop.lat)]
crs = {'init': 'EPSG:4326'}
geo_pop = gpd.GeoDataFrame(pop, crs=crs, geometry=geometry)

# geo_pop.to_file(driver='ESRI Shapefile', filename='geo_pop.shp')

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [8]:
geometry = [Point(xy) for xy in zip(des.longitude, des.latitude)]
crs = {'init': 'EPSG:4326'}
geo_des = gpd.GeoDataFrame(des, crs=crs, geometry=geometry)

# geo_pop.to_file(driver='ESRI Shapefile', filename='geo_des.shp')

### Use Google API to calculate travel distance and time

overview of the origin and destination data

In [9]:
geo_pop.head()

,objectid,lad19cd,lad19nm,lad19nmw,bng_e,bng_n,long,lat,st_areasha,st_lengths,geometry
0,1,E06000001,Hartlepool,NaN,447160,531474,-1.259401,54.669450,9.371262e+07,71011.933949,POINT (-1.25940 54.66945)
1,2,E06000002,Middlesbrough,NaN,451141,516887,-1.222262,54.542004,5.388156e+07,44481.691242,POINT (-1.22226 54.54200)
2,3,E06000003,Redcar and Cleveland,NaN,464361,519597,-1.020684,54.551652,2.450695e+08,96703.989701,POINT (-1.02068 54.55165)
3,4,E06000004,Stockton-on-Tees,NaN,444940,518183,-1.332280,54.561613,2.049330e+08,123408.985928,POINT (-1.33228 54.56161)
4,5,E06000005,Darlington,NaN,428029,515648,-1.552628,54.548705,1.974757e+08,107206.401677,POINT (-1.55263 54.54871)


In [10]:
geo_des.head()

,objectid,Region,STP,Trust or Site Name,Address,Postcode,longitude,latitude,category,object_id,...,Trust_or_S,Field9,Field10,Field11,STP_ICS,Centre,layer,path,new_id,geometry
0,1.0,South West,NHS Kernow CCG,Helston Medical Centre,"Trelawny Road, Helston, Cornwall",TR13 8AU,-5.269119,50.099159,GP_led vaccination services,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,POINT (-5.26912 50.09916)
1,2.0,South West,NHS Kernow CCG,Isles of Scilly Health Centre (deliver to main...,"The Health Centre, St Mary's, via Helston Medi...",TR21 0HE,-5.269998,50.099710,GP_led vaccination services,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,POINT (-5.27000 50.09971)
2,3.0,South West,NHS Kernow CCG,Falmouth Health Centre,"Falmouth Health Centre, Trescobeas Rd, Falmouth",TR11 2LH,-5.086106,50.155956,GP_led vaccination services,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-5.08611 50.15596)
3,4.0,South West,NHS Kernow CCG,Bodriggy Surgery,Bodriggy Health Centre,TR27 4PB,-5.412844,50.187164,GP_led vaccination services,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,POINT (-5.41284 50.18716)
4,5.0,South West,Cornwall and the Isles of Scilly,Truro Showground,"The Showground, Tubbin Hill, Stithians, Truro",TR3 7DP,-5.164745,50.194484,vaccination centres,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,POINT (-5.16474 50.19448)


In [11]:
# geo_pop = geo_pop[:50]
# geo_des = geo_des[:50]

Create a OD list. The list shows every OD pairs

In [12]:
from tqdm import tqdm
list_ori = []
list_ori_lat = []
list_ori_lon = []
list_des = []
list_des_lat = []
list_des_lon = []
for i in tqdm(range(1,geo_pop.shape[0]+1)):
    for j in range(geo_des.shape[0]):
        list_ori.append(i)
        list_ori_lat.append(geo_pop['lat'][i-1])
        list_ori_lon.append(geo_pop['long'][i-1])
        list_des.append(j)
        list_des_lat.append(geo_des['latitude'][j])
        list_des_lon.append(geo_des['longitude'][j])
od_cost = pd.DataFrame({'ori':list_ori,
                        'ori_lat':list_ori_lat,
                        'ori_lon':list_ori_lon,
                       'des':list_des,
                       'des_lat':list_des_lat,
                       'des_lon':list_des_lon,})

100%|██████████| 382/382 [00:48<00:00,  7.92it/s]


number of rows

In [13]:
od_cost.shape[0]

1330888

In [14]:
od_cost.head()

,ori,ori_lat,ori_lon,des,des_lat,des_lon
0,1,54.66945,-1.259401,0,50.099159,-5.269119
1,1,54.66945,-1.259401,1,50.099710,-5.269998
2,1,54.66945,-1.259401,2,50.155956,-5.086106
3,1,54.66945,-1.259401,3,50.187164,-5.412844
4,1,54.66945,-1.259401,4,50.194484,-5.164745


In [15]:
# geo_pop = geo_pop.to_crs(epsg = 27700)
# geo_des = geo_des.to_crs(epsg = 27700)
# distance_matrix = geo_pop.geometry.apply(lambda g: geo_des.distance(g))

In [16]:
import requests
import json
from math import sin, cos, sqrt, atan2, radians

calculate direct distance

In [17]:
def co_dis(ori_lat,ori_lng,des_lat,des_lng):
# approximate radius of earth in km
    R = 6373.0

    lat1 = radians(float(ori_lat))
    lon1 = radians(float(ori_lng))
    lat2 = radians(float(des_lat))
    lon2 = radians(float(des_lng))

    dlon = lon2 - lon1
    dlat = lat2 - lat1


    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * atan2(sqrt(a), sqrt(1 - a))


    return(R * c)

define a function to get the distance and time

In [18]:
def get_dis(ori_lat,
            ori_lng,
            des_lat,
            des_lng):
    
   
    # Take source as input
    source = str(ori_lng) + ',' + str(ori_lat)

    # Take destination as input
    dest = str(des_lng) + ',' + str(des_lat)

    # url variable store url 
    url ='http://router.project-osrm.org/route/v1/driving/'

    r = requests.get(url + source +
                       ';' + dest +
                       '?overview=false&steps=false')

    # json method of response object
    # return json format result
    try:
        x = r.json()
    except ConnectionError:
        return(np.nan,np.nan)
    
    # print(x)
    if x['code'] == 'InvalidQuery':
        return(np.nan,np.nan)


    # by default driving mode considered

    # get the travel distance in meters
    distance = x['routes'][0]['distance']
    # get the travel time in seconds
    travel_time = x['routes'][0]['duration']
    
    return(distance,travel_time)

In [19]:
# def get_dis(ori_lat,
#             ori_lng,
#             des_lat,
#             des_lng):
#     # enter your api key here
#     api_key =''

# #     ori_lat = ori['origin_y']
# #     ori_lng = ori['origin_x']
# #     des_lat = ori['latitude']
# #     des_lng = ori['longitude']
#     # Take source as input
#     source = str(ori_lat) + ',' + str(ori_lng)

#     # Take destination as input
#     dest = str(des_lat) + ',' + str(des_lng)

#     # url variable store url 
#     url ='https://maps.googleapis.com/maps/api/distancematrix/json?'

#     # Get method of requests module
#     # return response object
# #     print(url + 'origins=' + source +
# #                        '&destinations=' + dest +
# #                        '&key =' + api_key)
#     r = requests.get(url + 'origins=' + source +
#                        '&destinations=' + dest +
#                        '&key=' + api_key)

#     # json method of response object
#     # return json format result
#     x = r.json()

#     # by default driving mode considered

#     # get the travel distance in meters
# #     print(x['rows'][0]['elements'][0])
#     distance = x['rows'][0]['elements'][0]['distance']['value'] 
#     # get the travel time in seconds
#     travel_time = x['rows'][0]['elements'][0]['duration']['value'] 
#     return(distance,travel_time)

In [20]:
from tqdm import tqdm

list_dis = []
list_time = []
list_euc = []

In [21]:
for i,row in tqdm(od_cost.iterrows(), total=od_cost.shape[0]):
    ori_id = row['ori']
    des_id = row['des']

    if i < 1095868:
        continue
    
#     ori_lat = geo_pop.loc[geo_pop['OBJECTID'] == ori_id]['origin_y'].values[0]
#     ori_lng = geo_pop.loc[geo_pop['OBJECTID'] == ori_id]['origin_x'].values[0]
#     des_lat = geo_des.loc[geo_des['new_id'] == des_id]['latitude'].values[0]
#     des_lng = geo_des.loc[geo_des['new_id'] == des_id]['longitude'].values[0]

    ori_lat = row['ori_lat']
    ori_lng = row['ori_lon']
    des_lat = row['des_lat']
    des_lng = row['des_lon'] 
    
    euc_dis = co_dis(ori_lat,ori_lng,des_lat,des_lng)
#     print(euc_dis)
     
    if euc_dis > 25:
        list_dis.append( np.nan)
        list_time.append(np.nan) 
        list_euc.append(euc_dis) 
        continue
    
    distance,travel_time = get_dis(ori_lat,
                                    ori_lng,
                                    des_lat,
                                    des_lng)
    list_dis.append(distance)
    list_time.append(travel_time)
    list_euc.append(euc_dis) 

100%|██████████| 1330888/1330888 [35:41<00:00, 621.48it/s]    


In [24]:
i

1330887

In [23]:
# od_cost = od_cost.iloc[0:11]
od_cost['dis'] = list_dis
od_cost['time'] = list_time
od_cost['euc_dis'] = list_euc
od_cost.head()

ValueError: Length of values (235020) does not match length of index (1330888)

In [ ]:
od_cost.shape[0]

## Create the `access` Object

Since we have geometries here, it could be as simple as this:

In [ ]:
geo_des['sup'] = 10
geo_pop['All_Ages'] = 10

In [ ]:
A = Access(demand_df            = geo_pop,
           demand_index         = 'objectid',
           demand_value         = 'All_Ages',
           supply_df            = geo_des,
           supply_index         = 'new_id',
           supply_value         = ['sup'],
           cost_df              = od_cost,
           cost_origin          = 'ori',
           cost_dest            = 'des',
           cost_name            = 'time',
           neighbor_cost_df     = od_cost,
           neighbor_cost_origin = 'ori',
           neighbor_cost_dest   = 'des',
           neighbor_cost_name   = 'time')

### Add an Additional Distance Measure - Euclidean Distance

Though we've already loaded a network distance matrix, we can still add a Euclidean distance matrix. This requires geopandas to be installed, and you should already have projected the data. In case you have not, below we show how to see your current projection, and how to change it to the correct one (if you don't know what projection to use, you can reference the [spatial reference](https://spatialreference.org/) website).

In [ ]:
print(A.demand_df.crs)
print(A.supply_df.crs)

In [ ]:
A.demand_df = A.demand_df.to_crs(epsg = 27700)
A.supply_df = A.supply_df.to_crs(epsg = 27700)

Now we'll calculate the Euclidean distance matrix. Note that this is a very costly operation, but you only have to do it once.  `threshold` is the maximum distance (km) that we will consider. Calculating distances between centroids takes a while but is still faster than computing distances to polygons (crazy expensive, though we've been moderately careful to make it faster).

In [ ]:
A.create_euclidean_distance(threshold = 2500000, centroid_o = True, centroid_d = True)

Calling `A.cost_df` will now show the 'eulidean' column in the cost dataframe.

In [ ]:
A.cost_df

### Create Some Weight Functions

Of course, you don't _have_ to do this, but it's fun to play with different versions of spatial access, esp. E2SFCA. You can create the weighting functionsfrom stepwise weight dictionaries.

In [ ]:
fn30 = weights.step_fn({10 : 1, 20 : 0.68, 30 : 0.22})
fn60 = weights.step_fn({20 : 1, 40 : 0.68, 60 : 0.22})

Or, you can use gaussian weights.  Here, we're using a value of 20 (minutes), such that we'll be at the 3σ level at 60 minutes.

This could also be done through scipy.stats.norm().pdf, but that turns out to be unacceptably slow.

In [ ]:
gaussian = weights.gaussian(20)

Lastly, you could use a conventional gravity model, which is just weighted catchment with gravity weighting.

In [ ]:
gravity = weights.gravity(scale = 60, alpha = -1)

## Ready to roll...

Now just call the access functions. Each of these will return the corresponding series, but they are also stored in `A.access_df`.  Check out the documentation for the possible arguments. Note that if you call the same function repeatedly, with different arguments, you may want to change the name, to avoid overwriting. By default, `access` will calculate access metrics for each of the supply columns unless you specify a particular subset.

In [ ]:
A.weighted_catchment    (name = "gravity",  weight_fn = gravity)
A.fca_ratio             (name = "fca",      max_cost = 15)
A.fca_ratio             (name = "fca",      max_cost = 30) # Note - the warning -- good!
A.fca_ratio             (name = "fca60",    max_cost = 60)
A.fca_ratio             (name = "fca90",    max_cost = 90)
A.two_stage_fca         (name = "2sfca",    max_cost = 60)
A.enhanced_two_stage_fca(name = "2sfca30",  weight_fn = fn30)
A.enhanced_two_stage_fca(name = "2sfca60",  weight_fn = fn60)
A.enhanced_two_stage_fca(name = "g2sfca",   weight_fn = gaussian)
A.three_stage_fca       (name = "3sfca")

A.raam(name = "raam", tau = 60);
A.raam(name = "raam30", tau = 30);

In [ ]:
A.access_df.sort_values(by='raam_sup').dropna().head()

These were all with network travel time, in minutes.  We could also run the calculation with Euclidean distance.  Note that the parameters for the access fucntion will be dramatically different in meters instead of minutes!  Here, RAAM's tau parameter shifts from 60 minutes to 100 km (1e5 meters).

In [ ]:
raam = A.raam(name = "raam_euclidean", tau = 1e5, cost = "euclidean")

raam.head()

Gosh -- that's lots of access measures!  Let's simplify our life, with just one:

In [ ]:
A.score(name = "raam_combo", col_dict = {"raam_sup" : 1})

If there were only one cost function available, these logs wouldn't print.

If you found all the logs annoying instead of useful, then turn them off:

In [ ]:
A.log.setLevel(logging.WARNING)

## Results time!

Let's get the correlations among measures, and plot a few of them...

In [ ]:
A.norm_access_df.columns

In [ ]:
A.norm_access_df.corr().raam_sup.abs().sort_values(ascending = False)

In [ ]:
geo = gpd.read_file(Local_Authority_Districts_(December_2019)_Boundaries_UK_BFC.shp')
geo.head()

In [ ]:
A.norm_access_df

In [ ]:
A.norm_access_df.to_csv('figures/traveltime/metrics.csv')

In [ ]:
avg_traveltime = od_cost.groupby(['ori', 'ori_lat','ori_lon'], as_index=False).mean()
avg_traveltime = avg_traveltime[['ori', 'ori_lat','ori_lon','time']]
avg_traveltime.to_csv('figures/traveltime/avg_traveltime.csv')
# avg_traveltime

In [ ]:
il_map = geo.set_index('objectid').join(A.norm_access_df, how = "inner")

In [ ]:
il_map = il_map.join(avg_traveltime, how = "inner")

In [ ]:
def plot(v, ax = None, leg = True):
    
    swap_colors = "raam" in v
    
    il_map.plot(v, legend = leg, figsize = (5, 5), 
                cmap = "viridis_r" if swap_colors else "viridis", 
                markersize = 5, alpha = 0.8, ax = ax,
                vmin = il_map[v].quantile(0.05), vmax = il_map[v].quantile(0.95)).set_axis_off()

In [ ]:
plot("raam_sup")

In [ ]:
for name in A.norm_access_df.columns:
    plt.figure(figsize=(800, 600), dpi=1000)
    plot(name)
    plt.savefig('figures/traveltime/'+ name +'.png', dpi=1000)

In [ ]:
plt.figure(figsize=(800, 600), dpi=1000)
plot("time")
plt.savefig('figures/traveltime/平均时间.png', dpi=1000)